# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
city_data_df = pd.read_csv("data_frame.csv")
city_data_df.head()

,lat,lng,temp,hum,cloud,speed,city
0,21.4612,-71.1419,27.21,75,1,4.07,cockburn town
1,72.7868,-56.1549,-6.28,90,100,11.95,upernavik
2,9.0418,-69.7421,26.01,78,99,1.03,guanare
3,-38.0023,-57.5575,18.94,83,0,2.68,mar del plata
4,-53.1500,-70.9167,10.06,57,75,12.35,punta arenas


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
gmaps.configure(api_key='g_key')

In [19]:
lat_lon = city_data_df[['lat', 'lng']]

hum = city_data_df["hum"]

In [20]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(lat_lon, weights=hum, 
                                 dissipating=False, max_intensity=300,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
narrow_df = city_data_df.loc[(city_data_df['temp'] > 21) & (city_data_df['temp'] < 27) & (city_data_df['speed'] < 10) & (city_data_df['cloud'] ==0)].dropna()
                                                
narrow_df

,lat,lng,temp,hum,cloud,speed,city
32,-15.1961,12.1522,21.80,79,0,1.85,namibe
59,13.7539,-13.7586,25.03,41,0,1.03,tambacounda
98,-28.7667,114.6000,24.23,41,0,8.23,geraldton
129,9.4942,50.8122,24.55,63,0,7.70,bandarbeyla
135,-21.8269,-47.2486,23.63,78,0,6.69,santa cruz das palmeiras
274,-33.7456,-61.9688,22.65,45,0,4.45,venado tuerto
312,17.0151,54.0924,22.85,67,0,1.40,salalah
339,-16.7833,47.6500,21.70,66,0,4.91,tsaratanana
355,16.3324,-61.6979,25.02,88,0,2.06,sainte-rose
457,-17.8786,36.8883,26.97,88,0,3.13,quelimane


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = narrow_df[['lat', 'lng', 'city']].copy()

In [35]:
hotel_df['Hotel Df'] = ""

In [37]:
params = {'radius': 5000,
         'types': 'lodging',
         'key' : g_key}

for index, row in hotel_df.iterrows():
    lat = row["lat"]
    lng = row["lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = name_address['results'][0]['name']
    except (KeyError, IndexError):
        print("Missing")
        
hotel_df
   


Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing
Missing


,lat,lng,city,Hotel Df
32,-15.1961,12.1522,namibe,
59,13.7539,-13.7586,tambacounda,
98,-28.7667,114.6000,geraldton,
129,9.4942,50.8122,bandarbeyla,
135,-21.8269,-47.2486,santa cruz das palmeiras,
274,-33.7456,-61.9688,venado tuerto,
312,17.0151,54.0924,salalah,
339,-16.7833,47.6500,tsaratanana,
355,16.3324,-61.6979,sainte-rose,
457,-17.8786,36.8883,quelimane,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Df}</dd>
<dt>City</dt><dd>{city}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]


In [44]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)